## Generatore autocertificazioni

* Si parte da qui: https://github.com/GiovanniBalestrieri/autodichiarazione_2.0/blob/master/path_planning/basic_example.py

### Converto in funzione lo script della repo
        

In [2]:
def get_itinerary(trip):
    """
    This function takes a tuple of GPS positions tuples (lat, lon) and 
    returns openstreetmaps directions, in 
    terms of gps coordinates array.
    """
    
    import openrouteservice
    from openrouteservice import convert

    # Change your token here
    token = '5b3ce3597851110001cf62488a48a7f1e3ad491184353a3b070c32a2'
    
    # Connect to client
    client = openrouteservice.Client(key=token) # Specify your token

    # decode_polyline needs the geometry only
    geometry = client.directions(trip)['routes'][0]['geometry']
    decoded = convert.decode_polyline(geometry)

    return decoded['coordinates']

#trip = ((12.6067104,42.5917049), (12.6218790,42.5808930))
#directions = get_itinerary(trip)
#print(directions)

### Acquisizione coordinate

* I punti di interesse sono immessi manualmente, mentre i domicili sono generati randomicamente

* Il mumero di utenti in questo caso è uguale al numero di punti di partenza: ogni utente fa un trip

In [3]:
import numpy as np

n_users = 10           # Quanti utenti includere nella simulazione
delta_dist = 0.005      # Raggio della circonferenza che contiene i 
                       # punti di partenza (in gradi decimali)

dpi_list = ["mascherina", "guanti", "guanti e mascherina", "nessuno"]
simptoms_list = ["emicrania", "nausea", "dolori articolari", "respiro affannoso"]

interest_points = [{"name_loc": "CONAD Superstore", "addr": "Via del Rivo, 206, 05100 Terni TR, Italia", "lat":42.580256, "lon":12.621649, "cap": "05100", "type":"acquisti"},
                   {"name_loc": "Farmacia Nadalini", "addr": "Via del Rivo, 98-80, 05100 Terni TR, Italia", "lat":42.579970, "lon":12.625367, "cap": "05100", "type":"acquisti"},
                   {"name_loc": "Unicredit", "addr": "Via del Rivo, 208, 05100 Terni TR, Italia", "lat":42.580893, "lon":12.621879, "cap": "05100", "type":"banca"},
                   {"name_loc": "Ufficio", "addr": "Via del Camoscio, 19, 05100 Terni TR, Italia", "lat":42.582965, "lon":12.619052, "cap": "05100", "type":"ufficio"},
                   {"name_loc": "Ambulatorio", "addr": "Via del Rivo, 93, 05100 Terni TR, Italia", "lat":42.579069, "lon":12.626082, "cap": "05100", "type":"ambulatorio"}]

center_coords = (42.581302, 12.622702)

lats = list(np.random.uniform(center_coords[0]-delta_dist, center_coords[0]+delta_dist, n_users))
lons = list(np.random.uniform(center_coords[1]-delta_dist, center_coords[1]+delta_dist, n_users))
uuids = list(np.random.randint(low = 1000, high = 2000, size=n_users))

starting_points = [list(a) for a in zip(uuids, lats, lons)]

### Generazione trip

* Per ogni user, viene scelta una destinazione a caso

In [4]:
from datetime import datetime, timedelta 
from random import randint

raw_trips = []

for point in starting_points:

    trip = {}
    info = {}
    
    # Scelta destinazione per uuid corrente
    n = np.random.randint(len(interest_points))
    
    trip["uuid"] = int(point[0])
    trip["lat start"] = float(point[1])
    trip["lon start"] = float(point[2])
    trip["lat dest"] = interest_points[n]["lat"]
    trip["lon dest"] = interest_points[n]["lon"]
    trip["name dest"] = interest_points[n]["name_loc"]
    trip["address dest"] = interest_points[n]["addr"]
    trip["cap dest"] = interest_points[n]["cap"]
    trip["type dest"] = interest_points[n]["type"]
    
    # Generazione itinerario
    start_loc = (trip["lon start"], trip["lat start"])
    dest_loc = (trip["lon dest"], trip["lat dest"])
    trip_tuple = (start_loc, dest_loc)
    
    itinerary = get_itinerary(trip_tuple)
    trip["directions"] = list(itinerary)
    
    # Parametri utente
    temperature = np.random.uniform(35.5,37.5)
    simptoms = simptoms_list[np.random.randint(len(simptoms_list))]
    dpi = dpi_list[np.random.randint(len(dpi_list))]
    
    info["temperature"] = temperature
    info["simptoms"] = simptoms
    info["dpi"] = dpi
    
    start_time = datetime.now()
    
    # Add random time delta
    delta_h = 2
    delta_min = 5
    factor = randint(0, 3)
    start_time = start_time + timedelta(hours = delta_h*factor, minutes = delta_min*factor)
    
    info["start time"] = start_time.strftime("%m-%d-%Y_%H:%M:%S") 
    
    trip["info"] = info

    raw_trips.append(trip)

### Creazione dei geojson a partire dai dizionari dei trip

In [5]:
import json
from pprint import pprint

trips_dir = "trips/"

json_trips_list = []

for item in raw_trips:
    
    json_trip = json.dumps(item)
    json_trips_list.append(json_trip)
    
    pprint(json_trip)
    
    #filename = trips_dir + "trip_" + str(item["uuid"]) + ".geojson"
    
    #with open(filename, 'w') as json_file:
    #    json.dump(item, json_file)

('{"uuid": 1327, "lat start": 42.582297785154275, "lon start": '
 '12.619898141584683, "lat dest": 42.580256, "lon dest": 12.621649, "name '
 'dest": "CONAD Superstore", "address dest": "Via del Rivo, 206, 05100 Terni '
 'TR, Italia", "cap dest": "05100", "type dest": "acquisti", "directions": '
 '[[12.61992, 42.58238], [12.62055, 42.58228], [12.62052, 42.58218], '
 '[12.62051, 42.58214], [12.62044, 42.58192], [12.62039, 42.58173], [12.62034, '
 '42.58158], [12.62026, 42.58134], [12.62061, 42.58125], [12.62045, 42.58092], '
 '[12.62025, 42.58051], [12.62025, 42.58041], [12.6203, 42.58031], [12.62041, '
 '42.58023], [12.62051, 42.58017], [12.6207, 42.58038], [12.62079, 42.58034], '
 '[12.62134, 42.58004], [12.62145, 42.58008], [12.6216, 42.58006]], "info": '
 '{"temperature": 37.11567577892843, "simptoms": "nausea", "dpi": '
 '"mascherina", "start time": "03-31-2020_19:40:32"}}')
('{"uuid": 1946, "lat start": 42.577682859367414, "lon start": '
 '12.62399471741611, "lat dest": 42.579069,

### Upload to the Tangle

In [37]:
from iota import Iota, TryteString, Tag, ProposedTransaction
from iota.crypto.types import Seed
    
# Generate Seed and retrieve address
my_seed = Seed.random(length=81)
print('Your seed is: ' + str(my_seed))

# Declare an API object
api = Iota(
    adapter='https://nodes.devnet.iota.org:443',
    seed=my_seed,
    testnet=True,
)

print('Generating the first unused address...')
# Generate an address from your seed to post the transfer to
my_address = api.get_new_addresses(index=42)['addresses'][0]
print("Remember this address")
print(my_address)

print('Constructing transaction locally...')
    
def push_to_tangle(json_payload, my_address):
    """
    Pushing declaration to the distributed 
    digital ledger, the Tangle.
    iota >>> pip3 install pyota
    """

    # Convert to trytes
    trytes_data_user = TryteString.from_unicode(json_payload)

    # Tag is optional here
    my_tag = Tag(b'TESTINFORMATION')

    # Prepare a transaction object for user1
    tx_user = ProposedTransaction(
        address=my_address,
        value=0,
        tag=my_tag,
        message=trytes_data_user,
    )

    print('Sending transfer...')

    # Send the transaction to the network
    response = api.send_transfer([tx_user])

    print('Check your transaction on the Tangle for user')
    print('https://utils.iota.org/transaction/%s/devnet' % response['bundle'][0].hash)
    print('Tail transaction hash of the bundle is: %s' % response['bundle'].tail_transaction.hash)
    print('Done')
    
    return response['bundle'].tail_transaction.hash

Your seed is: MOXLMJMVETDTKOPTKRY9ZETNRLOSBGKWZEYIGRMK9LXIFWTLI9U9UYPDRLFBMUSDDGKBOL9QRSTTFD9QD
Generating the first unused address...
Remember this address
GUXQZUBPAADCYWOCIM9EMPWQDGAGBVNFKLCSROERRWTQDB9ZQVIZYIQNBA9UMQCMCKWOLKFCZAWZUTCLC
Constructing transaction locally...


In [39]:
list_hash = {}

idx = 0
for json_trip in json_trips_list:

    list_hash[str(idx)] = str(push_to_tangle(json_trip, my_address)) 
    #list_hash.append(str(push_to_tangle(json_trip, my_address)))
    idx = idx + 1

print(list_hash)
    
with open("hash_list.json", 'w') as json_file:
    #json_temp = json.dumps(list_hash)
    json.dump(list_hash, json_file)

Sending transfer...
Check your transaction on the Tangle for user
https://utils.iota.org/transaction/JTPRDPWQUJUJSKXENFWF9LYMWONSA9YQEUVGEPCZROABMHRUHFBWRAANOAZU9HPQBDXEVJVZAHIX9M999/devnet
Tail transaction hash of the bundle is: JTPRDPWQUJUJSKXENFWF9LYMWONSA9YQEUVGEPCZROABMHRUHFBWRAANOAZU9HPQBDXEVJVZAHIX9M999
Done
Sending transfer...
Check your transaction on the Tangle for user
https://utils.iota.org/transaction/BOLQMOAB9I9JXVFZXDIZAWBXENNFXLCIQFHXNGACMMH9WWCSVLMPARSHOCFGGJBSTCJQSAZSLDKS9E999/devnet
Tail transaction hash of the bundle is: BOLQMOAB9I9JXVFZXDIZAWBXENNFXLCIQFHXNGACMMH9WWCSVLMPARSHOCFGGJBSTCJQSAZSLDKS9E999
Done
Sending transfer...
Check your transaction on the Tangle for user
https://utils.iota.org/transaction/FXCQIYPBNYWLICWPTSFRPODMJCXAHDLDHZWZDOJ9FWVVGXHRIQRSVMGIWVXHKOYUQZJF9PVHRYXNIP999/devnet
Tail transaction hash of the bundle is: FXCQIYPBNYWLICWPTSFRPODMJCXAHDLDHZWZDOJ9FWVVGXHRIQRSVMGIWVXHKOYUQZJF9PVHRYXNIP999
Done
Sending transfer...
Check your transaction on the